In [ ]:
import pandas as pd

# 1) 원본 데이터 로드 (경로만 선현이형!!!!! 환경에 맞게 수정해주세요!!)
# file_path = r"원본파일경로.csv"
# df = pd.read_csv(file_path, encoding="utf-8-sig")

# 2) 이탈점수 계산에 사용할 컬럼 6개 정의
use_cols = []
use_cols.append("발급회원번호")
use_cols.append("이용금액_신용_B0M")
use_cols.append("잔액_B0M")
use_cols.append("이용건수_신용_B0M")
use_cols.append("기준년월")

# 3) 컬럼 존재 여부 체크
missing_cols = []
for col in use_cols:
    if col not in df.columns:
        missing_cols.append(col)

print("원본 컬럼 개수:", len(df.columns))
print("추출할 컬럼:", use_cols)
print("없는 컬럼:", missing_cols)

# 4) 없으면 여기서 멈추게 하기
if len(missing_cols) > 0:
    raise ValueError("원본 데이터에 필요한 컬럼이 없음: " + str(missing_cols))

# 5) 필요한 컬럼만 포함한 새로운 데이터프레임 생성
df_target_base = df[use_cols].copy()

# 6) 정렬(선택 사항이지만 보통 해두는 게 좋음)
df_target_base = df_target_base.sort_values(by=["발급회원번호", "기준년월"])

# 7) 인덱스 정리
df_target_base = df_target_base.reset_index(drop=True)

# 8) CSV 저장
output_path = "01_target_base_6cols.csv"
df_target_base.to_csv(output_path, index=False, encoding="utf-8-sig")

print("저장 완료:", output_path)
print("행 개수:", len(df_target_base))


In [ ]:
### 1번시트(카드 회원정보)에서 VIP/General 명단(txt) 만들기 ###

import pandas as pd
import os

# 0) 경로 설정 선현이형!!!!!!
#member_csv_path = r"data\01_카드회원정보.csv"
#output_dir = r"split_output"
#os.makedirs(output_dir, exist_ok=True)

# 1) 컬럼명 설정
id_col = "발급회원번호"
vip_col = "vip등급코드"

# 2) 필요한 컬럼만 로드(메모리 절약)
df_member = pd.read_csv(member_csv_path, usecols=[id_col, vip_col], encoding="utf-8-sig")

# 3) 발급회원번호 정리
df_member[id_col] = df_member[id_col].astype(str)
df_member[id_col] = df_member[id_col].str.strip()
df_member[id_col] = df_member[id_col].str.replace(".0", "", regex=False)

# 4) vip등급코드 정리
df_member[vip_col] = df_member[vip_col].astype(str)
df_member[vip_col] = df_member[vip_col].str.strip()

# 5) VIP 조건: "숫자"면 VIP
vip_mask = df_member[vip_col].str.isdigit()

# 6) VIP / General 분리
df_vip = df_member[vip_mask].copy()
df_general = df_member[~vip_mask].copy()

# 7) 2:8 비율 확인
total_cnt = len(df_member)
vip_cnt = len(df_vip)
general_cnt = len(df_general)
vip_ratio = 0
if total_cnt > 0:
    vip_ratio = vip_cnt / total_cnt

print("총:", total_cnt)
print("VIP:", vip_cnt)
print("GENERAL:", general_cnt)
print("VIP 비율(%):", round(vip_ratio * 100, 2))

# 8) ID 리스트 만들기(중복 제거)
vip_ids = df_vip[id_col].dropna().unique().tolist()
general_ids = df_general[id_col].dropna().unique().tolist()

# 9) txt 저장
vip_txt_path = os.path.join(output_dir, "vip_ids.txt")
general_txt_path = os.path.join(output_dir, "general_ids.txt")

with open(vip_txt_path, "w", encoding="utf-8") as f:
    for x in vip_ids:
        f.write(str(x) + "\n")

with open(general_txt_path, "w", encoding="utf-8") as f:
    for x in general_ids:
        f.write(str(x) + "\n")

print("저장 완료:", vip_txt_path)
print("저장 완료:", general_txt_path)


In [ ]:
### 2~8번 시트 CSV들을 VIP/GENERAL로 “쪼개서 저장” (대용량 안전 버전) ### 
import pandas as pd
import os

# 0) 경로 설정     선현이형!!!!!!!
#input_dir = r"data"
#output_dir = r"split_output"
#os.makedirs(output_dir, exist_ok=True)

# 1) 컬럼명 설정
id_col = "발급회원번호"

# 2) vip_ids.txt 로드
vip_txt_path = os.path.join(output_dir, "vip_ids.txt")

vip_list = []
with open(vip_txt_path, "r", encoding="utf-8") as f:
    for line in f:
        vip_list.append(line.strip())

vip_set = set(vip_list)
print("VIP ID 개수:", len(vip_set))

# 3) 분리할 파일 목록(2~8번 시트 파일명만 넣어)
target_files = []
target_files.append(r"02_시트.csv")
target_files.append(r"03_시트.csv")
target_files.append(r"04_시트.csv")
target_files.append(r"05_시트.csv")
target_files.append(r"06_시트.csv")
target_files.append(r"07_시트.csv")
target_files.append(r"08_시트.csv")

# 4) 큰 파일은 chunksize로 쪼개서 처리
chunksize = 200000

for filename in target_files:
    in_path = os.path.join(input_dir, filename)

    base_name = os.path.splitext(filename)[0]
    vip_out_path = os.path.join(output_dir, base_name + "_VIP.csv")
    gen_out_path = os.path.join(output_dir, base_name + "_GENERAL.csv")

    if os.path.exists(vip_out_path):
        os.remove(vip_out_path)

    if os.path.exists(gen_out_path):
        os.remove(gen_out_path)

    first_vip = True
    first_gen = True

    vip_rows = 0
    gen_rows = 0

    for chunk in pd.read_csv(in_path, chunksize=chunksize, encoding="utf-8-sig"):
        if id_col not in chunk.columns:
            raise ValueError("발급회원번호 컬럼이 없음: " + filename)

        chunk[id_col] = chunk[id_col].astype(str)
        chunk[id_col] = chunk[id_col].str.strip()
        chunk[id_col] = chunk[id_col].str.replace(".0", "", regex=False)

        vip_part = chunk[chunk[id_col].isin(vip_set)].copy()
        gen_part = chunk[~chunk[id_col].isin(vip_set)].copy()

        if len(vip_part) > 0:
            vip_part.to_csv(vip_out_path, index=False, encoding="utf-8-sig", mode="a", header=first_vip)
            first_vip = False
            vip_rows = vip_rows + len(vip_part)

        if len(gen_part) > 0:
            gen_part.to_csv(gen_out_path, index=False, encoding="utf-8-sig", mode="a", header=first_gen)
            first_gen = False
            gen_rows = gen_rows + len(gen_part)

    print("완료:", filename, "| VIP:", vip_rows, "| GENERAL:", gen_rows)
